In [1]:
import numpy as np
import open3d as o3d
from scipy.spatial.transform import Rotation as R

In [2]:
class Position():
    def __init__(self, array=[0.0,0.0,0.0]):
        self.x = array[0]
        self.y = array[1]
        self.z = array[2]


class Orientation():
    def __init__(self, quat=[0.0,0.0,0.0,1.0]):
        self.x = quat[0]
        self.y = quat[1]
        self.z = quat[2]
        self.w = quat[3]
        
class Pose:
    def __init__(self, position=Position(), orientation=Orientation()):
        self.position = position
        self.orientation = orientation

class Translation:
    def __init__(self, array=[0.0,0.0,0.0]):
        self.x = array[0]
        self.y = array[1]
        self.z = array[2]

class Quaternion:
    def __init__(self, quat=[0.0,0.0,0.0,1.0]):
        self.x = quat[0]
        self.y = quat[1]
        self.z = quat[2]
        self.w = quat[3]
    
class Transform:
    def __init__(self, translation=Translation(), quaternion=Quaternion()):
        self.translation = translation
        self.quaternion = quaternion

class Tf2_py:    
    @staticmethod
    def fromMsg(msg):
        T = np.eye(4)
        if msg.__class__.__name__ == 'Pose':
            t = [msg.position.x, msg.position.y, msg.position.z]
            quat = [msg.orientation.x, msg.orientation.y, msg.orientation.z, msg.orientation.w]

        elif msg.__class__.__name__ == 'Transform':
            t = [msg.translation.x, msg.translation.y, msg.translation.z]
            quat = [msg.quaternion.x, msg.quaternion.y, msg.quaternion.z, msg.quaternion.w]

        else:
            print("error")
        r = R.from_quat(quat).as_matrix()
        T[:3,:3] = r
        T[:3,3] = t
        
        return T
        
    @staticmethod
    def toPose(T):
        t = T[:3,3]
        r = T[:3,:3]
        quat = R.from_matrix(r).as_quat()
        position = Position(t)
        orientation = Orientation(quat)
        pose = Pose(position,orientation)
        
        return pose

    @staticmethod
    def toTransform(T):
        t = T[:3,3]
        r = T[:3,:3]
        quat = R.from_matrix(r).as_quat()
        translation = Translation(t)
        quaternion = Quaternion(quat)
        transform = Transform(translation, quaternion)
        
        return transform

In [3]:
R1 = R.from_euler('zxy', [np.pi/3, 0,0])
R2 = R.from_euler('zxy', [np.pi/6, 0,0])

In [4]:
R1.as_quat()

array([0.       , 0.       , 0.5      , 0.8660254])

In [5]:
translate1 = Translation([1,0,0])
quaternion1 = Quaternion(R1.as_quat())
transform1 = Transform(translate1, quaternion1)

In [6]:
translate2 = Translation([3,0,0])
quaternion2 = Quaternion(R2.as_quat())
transform2 = Transform(translate2, quaternion2)

In [7]:
T1 = Tf2_py.fromMsg(transform1)
T2 = Tf2_py.fromMsg(transform2)

In [8]:
transform = Tf2_py.toTransform(T1 @ T2)

In [9]:
vars(transform.translation), vars(transform.quaternion)

({'x': 2.500000000000001, 'y': 2.598076211353316, 'z': 0.0},
 {'x': 0.0, 'y': 0.0, 'z': 0.7071067811865474, 'w': 0.7071067811865477})

In [10]:
transform = Tf2_py.toTransform(T2 @ T1)

In [11]:
vars(transform.translation), vars(transform.quaternion)

({'x': 3.866025403784439, 'y': 0.49999999999999994, 'z': 0.0},
 {'x': 0.0, 'y': 0.0, 'z': 0.7071067811865474, 'w': 0.7071067811865477})

In [12]:
position1 = Position([1,0,0])
orientation1 = Orientation(R1.as_quat())
pose1 = Pose(position1, orientation1)

In [13]:
translate2 = Translation([3,0,0])
quaternion2 = Quaternion(R2.as_quat())
transform2 = Transform(translate2, quaternion2)

In [14]:
T1 = Tf2_py.fromMsg(pose1)
T2 = Tf2_py.fromMsg(transform2)

In [15]:
pose = Tf2_py.toPose(T1 @ T2)

In [16]:
vars(pose.position), vars(pose.orientation)

({'x': 2.500000000000001, 'y': 2.598076211353316, 'z': 0.0},
 {'x': 0.0, 'y': 0.0, 'z': 0.7071067811865474, 'w': 0.7071067811865477})

In [17]:
print((R.from_euler('zxy', [np.pi/2, 0,0])).as_quat())
print((R.from_euler('zxy', [np.pi/3, 0,0])).as_quat())
print((R.from_euler('zxy', [np.pi/6, 0,0])).as_quat())

[0.         0.         0.70710678 0.70710678]
[0.        0.        0.5       0.8660254]
[0.         0.         0.25881905 0.96592583]
